In [ ]:
import time
import pandas as pd

In [ ]:
# coding=utf-8
from pandas import read_csv
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
from pandas import concat
from sklearn.preprocessing import LabelEncoder


def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    print('待转换数据')
    print(df.head())
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        print('shift数据')
        print(cols[0][0:5])
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
        print('names数据')
        print(names[0:5])
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    print('拼接')
    print(agg[0:5])
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg



# print(values[0:5])


# encoder = LabelEncoder()
# values[:, 4] = encoder.fit_transform(values[:, 4])
# print('标签编码')
# print(values[0:5])

# values = values.astype('float32')
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(values)
# print('缩放')
# print(scaled[0:5])

# reframed = series_to_supervised(scaled, 1, 1)
# print('有监督')
# print(reframed[0:5])

# reframed.drop(reframed.columns[9:16], axis=1, inplace=True)
# print('删除不预测的列')
# print(reframed.head())


In [ ]:
def time2timestamp(timeStr):
    return int(time.mktime(time.strptime(timeStr, "%y-%m-%d %H:%M:%S")))


def labelEncoder(column):
    encoder = LabelEncoder()
    return encoder.fit_transform(column)

def printNoInstance(ls, type):
    for i, e in enumerate(ls):
        if not isinstance(e, type):
            # print(i, end=' ')
            try:
                type(e)
            except Exception as e:
                print("Error at {}: {}".format(i, e))


In [ ]:
full_data = read_csv("./error_full.csv")
data = full_data[["group", "报文时间", "车速", '总电压', '总电流', 'SOC（%）', 'DC-DC状态', '挡位', '电池单体电压最高值', '电池单体电压最低值', '最高温度值', '最低温度值', '最高报警等级', '加速度']]
data = data.drop(data[data["挡位"] == "停车P挡"].index)
data["报文时间"] = data["报文时间"].map(time2timestamp)
data = data[data["总电压"] != "异常"]
data = data.apply(pd.to_numeric, errors='ignore')
for k, v in dict(data.dtypes).items():
    if v == 'object':
        data[k] = labelEncoder(data[k])
data = data.astype('float64')

In [ ]:
data.dtypes


In [ ]:
if True not in data.isnull():
    data.to_csv("error_limit.csv")
else:
    print("error")


In [ ]:
for key in data.keys():
    # print(data[key].dtype)
    print("Key: {}".format(key))
    printNoInstance(list(data[key]), float)


In [ ]:
data.dtypes